<a href="https://colab.research.google.com/github/bill7845/colab_repo/blob/master/dacon_%EB%B8%8C%EB%9F%B0%EC%B9%98%EA%B8%80%EC%B6%94%EC%B2%9C/%EB%B8%8C%EB%9F%B0%EC%B9%98%EA%B8%80%EC%B6%94%EC%B2%9C_NOTE_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from collections import Counter
from datetime import timedelta, datetime
import glob
from itertools import chain
import json
import os
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
import seaborn as sns

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
# font_path = '/usr/share/fonts/NanumGothic.ttf'
# font_name = fm.FontProperties(fname=font_path, size=10).get_name()
# plt.rc('font', family=font_name, size=12)
# plt.rcParams["figure.figsize"] = (20, 10)
# register_matplotlib_converters()

In [0]:
directory = '/content/drive/My Drive/Colab data/카카오아레나_브런치데이터/'

**1. Data Read**

a. magazine

magazine.json: 매거진 정보



*   총 27,967개의 브런치 매거진 정보입니다.
*   필드 설명
    *   id : 매거진 식별자
    *   magazine_tag_list : 작가가 부여한 매거진의 태그정보

In [0]:
magazine = pd.read_json(directory + 'magazine.json', lines=True)

In [5]:
magazine.shape

(27967, 2)

In [6]:
magazine.head()

,magazine_tag_list,id
0,"[브런치북, 육아일기, 대화법, 들려주고픈이야기]",38842
1,"[tea, food]",11540
2,[food],11541
3,"[브런치북, 일상, 시, 사람]",11546
4,"[감성에세이, 노래, 음악에세이]",11544


b. Metadata.json

*   metadata.json : 글의 메타데이터

*   643,104 줄로 구성된 글의 메타데이터입니다.
*   이 메타데이터에는 2018년 10월 1일부터 2019년 3월 14일까지 독자들이 본 글에 대한 정보입니다.
*   작가가 비공개로 전환하였거나 삭제 등의 이유로 학습 데이터로 제공된 2018년 10월 1일부터 2019년 3월 1일 전까지의 본 글 정보에는 이 메타데이터에 없는 글이 있을 수 있습니다.
    *   개발 데이터와 평가 데이터에 포함된 글의 메타데이터도 포함되어 있습니다. 즉, 평가 대상자들이 2019년 3월 1일부터 2019년 3월 14일 사이에 본 모든 글에 대한 정보가 포함되어 있습니다.

*   필드설명
    *   magazine_id: 이 글의 브런치 매거진 아이디 (없을 시는 0)
    *   reg_ts: 이 글이 등록된 시간(유닉스 시간, 밀리초)
    *   user_id: 작가 아이디
    *   article_id: 글 번호
    *   id: 글 식별자
    *   title: 제목
    *   sub_title: 부제목
    *   display_url: 웹 주소
    *  keyword_list: 작가가 부여한 글의 태그 정보
*   메타데이터의 모든 정보는 작가의 비공개 여부 전환, 글 삭제, 수정 등으로 유효하지 않거나 변동될 수 있습니다.












In [0]:
metadata = pd.read_json(directory + 'metadata.json', lines=True)

In [8]:
metadata.shape

(643104, 9)

In [9]:
metadata.head()

,magazine_id,user_id,title,keyword_list,display_url,sub_title,reg_ts,article_id,id
0,8982,@bookdb,"사진으로 옮기기에도 아까운, 리치필드 국립공원","[여행, 호주, 국립공원]",https://brunch.co.kr/@bookdb/782,세상 어디에도 없는 호주 Top 10,1474944427000,782,@bookdb_782
1,12081,@kohwang56,[시] 서러운 봄,"[목련꽃, 아지랑이, 동행]",https://brunch.co.kr/@kohwang56/81,,1463092749000,81,@kohwang56_81
2,0,@hannahajink,무엇을 위해,[],https://brunch.co.kr/@hannahajink/4,무엇 때문에,1447997287000,4,@hannahajink_4
3,16315,@bryceandjuli,싫다,"[감정, 마음, 위로]",https://brunch.co.kr/@bryceandjuli/88,,1491055161000,88,@bryceandjuli_88
4,29363,@mijeongpark,Dubliner#7,"[유럽여행, 더블린, 아일랜드]",https://brunch.co.kr/@mijeongpark/34,#7. 내 친구의 집은 어디인가,1523292942000,34,@mijeongpark_34


c. Users.json

*   users.json: 사용자 정보
    *   가입한 사용자(작가 혹은 독자)의 정보입니다.
    * 필드 설명
        *   keyword_list: 최근 며칠간 작가 글로 유입되었던 검색 키워드
        *   following_list: 구독 중인 작가 리스트
        *   id: 사용자 식별자
    *   총 310,758명의 정보가 있습니다. 탈퇴 등의 이유로 사용자 정보가 없을 수 있습니다.


In [0]:
users = pd.read_json(directory + '/users.json', lines=True)

In [11]:
users.shape

(310758, 3)

In [12]:
users.head()

,keyword_list,following_list,id
0,[],"[@perytail, @brunch]",#901985d8bc4c481805c4a4f911814c4a
1,[],"[@holidaymemories, @wadiz, @sciforus, @dailydu...",#1fd89e9dcfa64b45020d9eaca54e0eed
2,[],"[@commerceguy, @sunsutu, @kakao-it, @joohoonja...",#1d94baaea71a831e1f33e1c6bd126ed5
3,[],"[@amberjeon48, @forsy20, @nemotokki, @hawann, ...",#04641c01892b12dc018b1410e4928c0d
4,[],"[@dwcha7342, @iammento, @kakao-it, @dkam, @ant...",#65bcaff862aadff877e461f54187ab62


d. Read Files


*   read 디렉토리: 본 글 정보
    *   read.tar은 2018년 10월 1일부터 2019년 3월 1일까지 일부 브런치 독자들이 본 글의 정보가 총 3,625개의 파일로 구성되어 있습니다.
    *  파일의 이름은 시작일_종료일 형태입니다. 예를 들어 2018110708_2018110709 파일은 2018년 11월 7일 오전 8시부터 2018년 11월 7일 오전 9시 전까지 본 글입니다.
    *   파일은 여러 줄로 이뤄져 있으며 하나의 줄은 브런치의 독자가 파일의 시간 동안 본 글을 시간 순으로 기록한 것입니다. 한 줄의 정보는 공백으로 구분되어 있으며 첫번 째가 독자의 암호화된 식별자이고 그 뒤로는 해당 독자가 본 글의 정보입니다.
    *   예를 들어 read/2019022823_2019030100 파일에 기록된 아래 정보는 #8a706ac921a11004bab941d22323efab 라는 독자가 2019년 2월 28일 23시에서 2019년 3월 1일 0시 사이에 @bakchacruz_34 @wo-motivator_133 @wo-motivator_133 를 순서대로 보았다는 뜻입니다. @wo-motivator_133 글이 두 번 나타난 것은 이 글을 보기 위해 두 번 방문했다는 뜻입니다.
          * **`#8a706ac921a11004bab941d22323efab @bakchacruz_34 @wo-motivator_133 @wo-motivator_133`**
    *   글을 보았다는 의미는 특정 글에 모바일, PC, 앱을 통해 접근했다는 뜻입니다. 머문 시간에 대한 정보가 제공되지 않기 때문에 실제로 글을 읽지 않고 이탈하는 등의 가능성도 있습니다.







In [0]:
# glob.glob(path) # path 내에 모든 파일을 '경로+파일명'의 list 형식으로 가져옴
read_file_lst = glob.glob(directory + 'read/*')

In [0]:
exclude_file_lst = ['read.tar']

In [0]:
read_df_lst = []
for f in read_file_lst:
    file_name = os.path.basename(f) # os.path.basename('name') => 해당 이름을 가진 폴더/파일 
    if file_name in exclude_file_lst:
        print(file_name)
    else:
        df_temp = pd.read_csv(f, header=None, names=['raw'])
        df_temp['dt'] = file_name[:8] # file_name에서 datetime 해당부분만
        df_temp['hr'] = file_name[8:10] # file_name에서 hr 해당부분만
        df_temp['user_id'] = df_temp['raw'].str.split(' ').str[0] # raw 에서 user_id 추출
        df_temp['article_id'] = df_temp['raw'].str.split(' ').str[1:].str.join(' ').str.strip() # article_id 추출
        read_df_lst.append(df_temp)

In [0]:
read = pd.concat(read_df_lst)

In [0]:
read.to_csv(directory + 'read.csv', index=None)

In [0]:
read.head()

In [0]:
a = pd.read_csv(directory + 'read.csv')

In [37]:
s = a[a.dt == 20181001]
s[s.hr == 00]

,raw,dt,hr,user_id,article_id
3119762,#e208be4ffea19b1ceb5cea2e3c4dc32c @kty0613_91,20181001,0,#e208be4ffea19b1ceb5cea2e3c4dc32c,@kty0613_91
3119763,#0a3d493f3b2318be80f391eaa00bfd1c @miamiyoung_...,20181001,0,#0a3d493f3b2318be80f391eaa00bfd1c,@miamiyoung_31 @banksalad_49 @rlfrjsdn_95 @rea...
3119764,#b90d3ee7ed0d7d827aae168e159749f1 @joeunha_4 @...,20181001,0,#b90d3ee7ed0d7d827aae168e159749f1,@joeunha_4 @yoonvi_3
3119765,#b8b9d09fe2961fd62edc94912bf75a90 @hyejinchoi_...,20181001,0,#b8b9d09fe2961fd62edc94912bf75a90,@hyejinchoi_122 @hyejinchoi_86 @hyejinchoi_42 ...
3119766,#072f742eda9359cdac03ad080193c11d @doyeonsunim...,20181001,0,#072f742eda9359cdac03ad080193c11d,@doyeonsunim_240 @k52524_297 @bule13_33 @kwong...
...,...,...,...,...,...
3120982,#7c5821cc6cd830702a8b2cc5c9972d79 @alicemelbou...,20181001,0,#7c5821cc6cd830702a8b2cc5c9972d79,@alicemelbourne_174 @alicemelbourne_179 @alice...
3120983,#786c0a67798046548075d93df609b862 @missingmay_...,20181001,0,#786c0a67798046548075d93df609b862,@missingmay_141
3120984,#55e9271a52a4de061771b55aec78adc6 @leejarak_19...,20181001,0,#55e9271a52a4de061771b55aec78adc6,@leejarak_198 @leejarak_201 @leejarak_198
3120985,#c5b67e1350aa943bbe0acdeccbc452a5 @tenbody_1422,20181001,0,#c5b67e1350aa943bbe0acdeccbc452a5,@tenbody_1422
